In [17]:
import pandas as pd
import numpy as np
import scipy.stats as st
import math
import xgboost as xgb
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

In [45]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

In [46]:
train.head()
train_all = train.drop(labels= ["Stock_ID","timestamp","ID","Outcome"], axis=1)
test_all = test.drop(labels= ["Stock_ID","timestamp","ID"], axis=1)

In [47]:
test_all.head()

,Average_True_Range,Five_Day_Moving_Average,Negative_Directional_Movement,Positive_Directional_Movement,Ten_Day_Moving_Average,Three_Day_Moving_Average,True_Range,Twenty_Day_Moving_Average,Volume
0,0.17058,-1.08146,-0.33365,-0.39903,-1.02367,-1.09885,0.11595,-0.95527,0.43959
1,-0.48696,-0.64340,-0.30166,-0.28545,-0.63477,-0.64131,-0.47816,-0.61062,-0.38494
2,0.86787,-0.46261,-0.43770,-0.31078,-0.22662,-0.53956,0.27538,-0.15185,1.38417
3,0.04259,-1.66702,-0.31910,-0.24140,-1.66951,-1.64317,-0.51638,-1.58663,-0.22040
4,-0.63238,0.12088,-0.45965,-0.40673,0.15728,0.14539,0.10491,0.13933,-0.21500


In [30]:
#Always call fit on the estimator before invoking this method.
def importances(estimator, col_array, title):

# Calculate the feature ranking - Top 10
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1] #It returns the indices that would sort an array.

    print "%s Top 20 Important Features\n" %title

    for f in range(10):
        print("%d. %s   (%f)" % (f + 1, col_array.columns[indices[f]], importances[indices[f]]))
    
#Mean Feature Importance
    print "\nMean Feature Importance %.6f" %np.mean(importances)
    
#Plot the feature importances of the forest
    indices=indices[:10]
    plt.figure()
    plt.title(title+" Top 10 Feature importances")
    plt.bar(range(10), importances[indices],
            color="gr", align="center")
    plt.xticks(range(10), col_array.columns[indices], fontsize=14, rotation=90)
    plt.xlim([-1, 10])
    plt.show()

In [19]:
train.head()

,Average_True_Range,Five_Day_Moving_Average,ID,Negative_Directional_Movement,Outcome,Positive_Directional_Movement,Stock_ID,Ten_Day_Moving_Average,Three_Day_Moving_Average,True_Range,Twenty_Day_Moving_Average,Volume,timestamp
0,-0.38778,-0.98736,1_1,-0.14321,1.0,-0.39903,1,-0.94728,-0.99681,-0.21129,-0.91977,-0.33308,1
1,0.97308,-2.08251,3_1,0.80286,0.0,-0.31078,3,-2.02183,-2.08895,-0.46476,-1.93779,-0.67096,1
2,-0.52368,1.09143,4_1,-0.31910,0.0,-0.24140,4,1.18737,1.07839,-0.51638,0.21868,-0.22040,1
3,-0.69311,-0.53976,6_1,-0.45965,0.0,-0.40673,6,-0.47569,-0.55106,-0.60816,-0.42449,-0.23118,1
4,-0.43690,-1.39805,10_1,0.29217,0.0,-0.33347,10,-1.36756,-1.40937,-0.33206,-1.31365,0.15076,1


In [20]:
train.isnull()

,Average_True_Range,Five_Day_Moving_Average,ID,Negative_Directional_Movement,Outcome,Positive_Directional_Movement,Stock_ID,Ten_Day_Moving_Average,Three_Day_Moving_Average,True_Range,Twenty_Day_Moving_Average,Volume,timestamp
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False


In [9]:
train['mean_moving'] = (train['Three_Day_Moving_Average'] + train['Five_Day_Moving_Average'] + train['Ten_Day_Moving_Average'] + train['Twenty_Day_Moving_Average'])/4
train1 = train[['Stock_ID','Volume','True_Range','Average_True_Range','Positive_Directional_Movement','Negative_Directional_Movement','mean_moving']]

In [10]:
train1.head()

,Stock_ID,Volume,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement,mean_moving
0,1,-0.33308,-0.21129,-0.38778,-0.39903,-0.14321,-0.962805
1,3,-0.67096,-0.46476,0.97308,-0.31078,0.80286,-2.032770
2,4,-0.22040,-0.51638,-0.52368,-0.24140,-0.31910,0.893967
3,6,-0.23118,-0.60816,-0.69311,-0.40673,-0.45965,-0.497750
4,10,0.15076,-0.33206,-0.43690,-0.33347,0.29217,-1.372157


In [11]:
test1 = test
test1['mean_moving'] = (test1['Three_Day_Moving_Average'] + test1['Five_Day_Moving_Average'] + test1['Ten_Day_Moving_Average'] + test1['Twenty_Day_Moving_Average'])/4
test1 = test1[['Stock_ID','Volume','True_Range','Average_True_Range','Positive_Directional_Movement','Negative_Directional_Movement','mean_moving']]

In [13]:
test1.head()

,Stock_ID,Volume,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement,mean_moving
0,1,0.43959,0.11595,0.17058,-0.39903,-0.33365,-1.039813
1,2,-0.38494,-0.47816,-0.48696,-0.28545,-0.30166,-0.632525
2,3,1.38417,0.27538,0.86787,-0.31078,-0.43770,-0.345160
3,4,-0.22040,-0.51638,0.04259,-0.24140,-0.31910,-1.641583
4,6,-0.21500,0.10491,-0.63238,-0.40673,-0.45965,0.140720


In [48]:
import xgboost as xgb
xgb1 = xgb.XGBClassifier(
        learning_rate =0.1,
        n_estimators=50,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb1.fit(train_all,train['Outcome'])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=50, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [49]:
temp = pd.DataFrame()
temp['ID'] = test['ID']
temp['Outcome'] = xgb1.predict(test_all)
temp.to_csv('xgb5.csv', header=True, index = False)

In [50]:
xgb1.predict(test_all).mean()

0.10837109842465618

In [51]:
importances(xgb1, train_all, "Cover Type (Initial RF)")

Cover Type (Initial RF) Top 20 Important Features

1. Positive_Directional_Movement   (0.176707)
2. Volume   (0.143240)
3. Negative_Directional_Movement   (0.134538)
4. True_Range   (0.131861)
5. Average_True_Range   (0.105756)
6. Twenty_Day_Moving_Average   (0.093708)
7. Three_Day_Moving_Average   (0.079652)
8. Five_Day_Moving_Average   (0.078313)
9. Ten_Day_Moving_Average   (0.056225)


IndexError: index 9 is out of bounds for axis 0 with size 9

In [24]:
train['mean_moving'] = (train['Three_Day_Moving_Average'] + train['Five_Day_Moving_Average'] + train['Ten_Day_Moving_Average'] + train['Twenty_Day_Moving_Average'])/4
train2 = train[['Volume','True_Range','Positive_Directional_Movement','Negative_Directional_Movement','mean_moving']]

In [25]:
test1 = test
test1['mean_moving'] = (test1['Three_Day_Moving_Average'] + test1['Five_Day_Moving_Average'] + test1['Ten_Day_Moving_Average'] + test1['Twenty_Day_Moving_Average'])/4
test2 = test1[['Volume','True_Range','Positive_Directional_Movement','Negative_Directional_Movement','mean_moving']]

In [26]:
xgb1.fit(train2,train['Outcome'])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=50, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [28]:
temp = pd.DataFrame()
temp['ID'] = test['ID']
temp['Outcome'] = xgb1.predict(test2)
temp.to_csv('xgb4.csv', header=True, index = False)

In [54]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt=GradientBoostingClassifier(n_estimators=50)
gbrt.fit(train_all,train['Outcome'])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=50, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [55]:
temp = pd.DataFrame()
temp['ID'] = test['ID']
temp['Outcome'] = gbrt.predict(test_all)
temp.to_csv('xgb4.csv', header=True, index = False)